In [ ]:
import pandas as pd
from transformers import AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
# Load the dataset
file_path = '/content/drive/MyDrive/Combined Data.csv'
df = pd.read_csv(file_path)

In [ ]:
target_column = 'status'
statement_column = 'statement'
df[statement_column] = df[statement_column].astype(str)


In [ ]:

# Get the count of rows for each class
class_counts = df[target_column].value_counts()

print(class_counts)


status
Normal                  16351
Depression              15404
Suicidal                10653
Anxiety                  3888
Bipolar                  2877
Stress                   2669
Personality disorder     1201
Name: count, dtype: int64


In [ ]:
# Define the number of samples you want for 'Normal', 'Depression', and 'Suicidal'
normal_count = 5000  # Target number of rows for 'Normal'
depression_count = 5000  # Target number of rows for 'Depression'
suicidal_count = 5000

In [ ]:
# Filter out rows where the 'statement' column exceeds 1024 tokens
def filter_statements_by_token_length(df, max_tokens=1024):
    def is_below_token_limit(statement):
        tokenized = tokenizer(statement, truncation=False)['input_ids']
        return len(tokenized) <= max_tokens

    return df[df[statement_column].apply(is_below_token_limit)]

# Apply the filtering to the whole dataset
df_filtered_by_tokens = filter_statements_by_token_length(df)

# Get the count of rows for each class
class_counts = df_filtered_by_tokens[target_column].value_counts()
print(class_counts)

# Split the dataset by class
normal_df = df_filtered_by_tokens[df_filtered_by_tokens[target_column] == 'Normal']
depression_df = df_filtered_by_tokens[df_filtered_by_tokens[target_column] == 'Depression']
suicidal_df = df_filtered_by_tokens[df_filtered_by_tokens[target_column] == 'Suicidal']
other_df = df_filtered_by_tokens[~df_filtered_by_tokens[target_column].isin(['Normal', 'Depression', 'Suicidal'])]

# Randomly sample from 'Normal', 'Depression', and 'Suicidal' to reduce their size
normal_sampled = normal_df.sample(n=normal_count, random_state=42)
depression_sampled = depression_df.sample(n=depression_count, random_state=42)
suicidal_sampled = suicidal_df.sample(n=suicidal_count, random_state=42)

# Combine the sampled 'Normal', 'Depression', 'Suicidal', and other classes
df_reduced = pd.concat([normal_sampled, depression_sampled, suicidal_sampled, other_df])

# Shuffle the resulting dataframe
df_reduced = df_reduced.sample(frac=1, random_state=42).reset_index(drop=True)
print(df_reduced.shape)

status
Normal                  16351
Depression              15236
Suicidal                10559
Anxiety                  3863
Bipolar                  2852
Stress                   2660
Personality disorder     1190
Name: count, dtype: int64
(25565, 3)


In [ ]:


# Remove rows where the target class is 'Bi-Polar' or 'Personality Disorder'
df_filtered = df_reduced[~df_reduced[target_column].isin(['Bipolar', 'Personality disorder'])]

# Shuffle the dataset
df_shuffled = df_filtered.sample(frac=1).reset_index(drop=True)

In [ ]:
# Save the shuffled dataset to a new CSV file
output_file = 'filtered_shuffled_dataset.csv'
df_shuffled.to_csv(output_file, index=False)


In [ ]:
print(df_shuffled.head())
print(df_shuffled.tail())
print(df_shuffled.shape)

   Unnamed: 0                                          statement      status
0       52646  Anxious habit help Hi everyone,\n\nI have alwa...     Anxiety
1       31334                               where's the mustard?      Normal
2       24386  I have left school now and I am moving onto a ...  Depression
3        1205  Kuy- wtb pc haechan is smooth or damage is thi...      Normal
4        9573  Betrayed and f*cked over by everyone, been try...    Suicidal
       Unnamed: 0                                          statement  \
21518       45312  absolutelybatty hug back thanks hon i can t be...   
21519       38031  i didn t do the best in school i doubt i ll ha...   
21520       20134  I wish I would die or someone else would take ...   
21521       29330  BACKGROUND: I didn't know much about PTSD when...   
21522       32791  it's the law. they can't keep him in jail fore...   

           status  
21518      Normal  
21519  Depression  
21520  Depression  
21521      Normal  
21522

In [ ]:
# save to output data folder
!cp filtered_shuffled_dataset.csv /content/drive/MyDrive